In [1]:
import os
import pandas as pd
import re
import numpy as np
import datetime

In [2]:
itemList = ["塞德思","灵的融","希纳露","百热朗","健眠计划","S+","琵力消","VL"]
def tag_add(itemName,itemLists):
    for item in itemLists:
        if item in itemName:
            return item
        else:
            continue
    return itemName

In [3]:
bilibili_list = [730603253025,730901991727,730903103301]
def bilibili_taoke_divide(sku_id,bilibili_list,file_name):
    if sku_id in bilibili_list and ("定向计划" in file_name or "自主推广-商品维度-推广概览-数据分布" in file_name):
        return "B站"
    else:
        return "taoke"

In [4]:
csv_list=os.listdir(r"D:\盐野义\月报\2024年7月\淘客数据")
taoke_data = pd.read_csv("D:/盐野义/月报/2024年7月/淘客数据/"+csv_list[0])
taoke_data["渠道"] = taoke_data["商品ID"].apply(lambda x:bilibili_taoke_divide(x,bilibili_list=bilibili_list,file_name=csv_list[0]))

for csvs in csv_list[1:]:    
    taoke_data_sub = pd.read_csv("D:/盐野义/月报/2024年7月/淘客数据/"+csvs)
    taoke_data_sub["渠道"] = taoke_data["商品ID"].apply(lambda x:bilibili_taoke_divide(x,bilibili_list=bilibili_list,file_name=csvs))
    taoke_data = pd.concat([taoke_data,taoke_data_sub],axis=0,ignore_index=True)
taoke_data["商品别名"] = taoke_data["商品名称"].astype(str).apply(lambda x:tag_add(x,itemLists=itemList))


In [5]:
taoke_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   商品ID           80 non-null     int64  
 1   商品名称           73 non-null     object 
 2   商品链接           73 non-null     object 
 3   点击量(即进店量)      80 non-null     int64  
 4   点击人数(即进店人数)    80 non-null     int64  
 5   点击转化率(即付款转化率)  46 non-null     object 
 6   添加购物车量         80 non-null     int64  
 7   收藏宝贝量          80 non-null     int64  
 8   付款人数           41 non-null     float64
 9   付款金额           80 non-null     float64
 10  付款笔数           80 non-null     int64  
 11  付款件数           80 non-null     int64  
 12  付款佣金支出         80 non-null     float64
 13  付款服务费支出        80 non-null     float64
 14  付款佣金率          41 non-null     object 
 15  付款服务费率         41 non-null     object 
 16  付款支出费用         80 non-null     float64
 17  结算人数           80 non-null     int64  
 18  结算笔数        

In [8]:
taoke_data_pivot = taoke_data.pivot_table(values=["点击量(即进店量)","付款人数","付款支出费用","付款金额"],index=["渠道","商品别名"],aggfunc=[np.sum])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7948\3755354769.py:1: FutureWarning: The provided callable <function sum at 0x0000015443396CA0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taoke_data_pivot = taoke_data.pivot_table(values=["点击量(即进店量)","付款人数","付款支出费用","付款金额"],index=["渠道","商品别名"],aggfunc=[np.sum])


In [9]:
taoke_data_pivot.to_excel(r"D:\盐野义\月报\2024年7月\淘客数据\7月淘客总结.xlsx")